In [1]:
pip install newsapi

In [2]:
import requests
 
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
def analyze_news_sentiment(stock_symbol, base_url="https://finance.yahoo.com/quote/"):
  """
  Attempts to analyze sentiment of news related to the stock symbol
  by scraping a finance website (use with caution due to scraping limitations).

  Args:
      stock_symbol: The stock ticker symbol (e.g., "AAPL").
      base_url: The base URL of the finance website (default: Yahoo Finance).

  Returns:
      A list of tuples containing the scraped headline (if found)
      and its sentiment analysis (positive, neutral, negative).
  """
  url = f"{base_url}{stock_symbol}/news"
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for failed requests
  except requests.exceptions.RequestException as e:
    print(f"Error: Failed to retrieve news from {url} ({e})")
    return []

  soup = BeautifulSoup(response.content, 'html.parser')
  news_items = soup.find_all('li', class_="news-item")  # Adjust selectors based on website structure
  sentiments = []
  analyzer = SentimentIntensityAnalyzer()
  for item in news_items:
    headline = item.find('a', class_="news-item__ headline")  # Adjust selectors for headline elements
    if headline:
      headline_text = headline.text.strip()
      scores = analyzer.polarity_scores(headline_text)
      compound_score = scores["compound"]
      sentiment = determine_sentiment(compound_score)
      sentiments.append((headline_text, sentiment))
  return sentiments

def determine_sentiment(compound_score):
  """
  Determines sentiment based on the compound score from VADER.

  Args:
      compound_score: The compound sentiment score from VADER analysis.

  Returns:
      A string representing the sentiment (positive, neutral, negative).
  """
  if compound_score >= 0.05:
    return "Positive"
  elif compound_score <= -0.05:
    return "Negative"
  else:
    return "Neutral"

def main():
  """
  Main function to perform sentiment analysis on scraped news (use with caution).
  """
  stock_symbol = input("Enter the stock symbol (e.g., AAPL): ")
  sentiments = analyze_news_sentiment(stock_symbol)

  if sentiments:
    print(f"Sentiment Analysis for News related to {stock_symbol} (scraped from Yahoo Finance)")
    for headline, sentiment in sentiments:
      print(f"- '{headline}': {sentiment}")
  else:
    print(f"No news articles found for {stock_symbol} on Yahoo Finance.")

if __name__ == "__main__":
  main()


Enter the stock symbol (e.g., AAPL): aapl
Error: Failed to retrieve news from https://finance.yahoo.com/quote/aapl/news (404 Client Error: Not Found for url: https://finance.yahoo.com/quote/aapl/news)
No news articles found for aapl on Yahoo Finance.
